# find (+follow) face

In [1]:
import sys
# customize path according to where sdk repo is in your system
sdk_path = '/home/david/GIT/cozmo-python-sdk/examples/tutorials/03_vision/'
sys.path.append(sdk_path)
from face_follow import follow_faces

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/home/david/anaconda3/envs/cozmo/lib/python3.7/site-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


In [2]:
sys.path.append('/home/david/GIT/cozmo-python-sdk/')
sys.path.append('/home/david/GIT/cozmo-python-sdk/src')
sys.path.append('/home/david/GIT/cozmo-python-sdk/src/cozmo')
from src import cozmo

In [3]:
# cozmo.run_program(follow_faces, use_viewer=True, force_viewer_on_top=True)

# find object

* How to get bounding box from stream. 

One at a time. 

Don't use a thread?

* How to make it discrete?

In [4]:
behavior='LookAroundInPlace'
be_id=6

face_be='FindFaces'
faceBe_id=1

In [5]:
sys.path.append('/home/david/GIT/language-acquisition/')

In [7]:
import time
from classes.vision import Vision

def take_picture(robot):
    look = Sight(robot)
    robot.set_lift_height(1.0).wait_for_completed()
    robot.set_head_angle(cozmo.robot.MIN_HEAD_ANGLE).wait_for_completed()
    exposure_amount = 0.1 # Range: [0,1]
    gain_amount = 1.0 # Range: [0,1]
    color = False
    look.configure_camera(robot, exposure_amount, gain_amount, color)    
    robot.camera.color_image_enabled = True
    robot.camera.image_stream_enabled = True
    latest_image = robot.world.latest_image
    if latest_image is not None:
        annotated = latest_image.annotate_image()
        print("image = %s" % latest_image)
        print('Detect object? (y for yes, s for skip):')
        response = input()
        if response.strip() == 'y':
            annotated = latest_image.annotate_image()
            print('annotated type',type(annotated))
            obj = look.disc_detect_obj(annotated)
    return obj

In [9]:
from classes.vision import Vision
from matplotlib import pyplot as plt
from PIL import Image
import datetime
import cv2
import numpy as np

from src.cozmo.util import degrees, Angle, distance_mm, speed_mmps

def cozmo_program(robot: cozmo.robot.Robot):
    robot.clear_idle_animation()
    x_centered = False
    y_centered = False
    while not x_centered and not y_centered:
        obj = take_picture(robot)
        boxes = obj['detection_boxes']    
        go_box = boxes[0]
#         print('box in question',go_box)
        ymin = go_box[0]
        ymax = go_box[1]
        xmin = go_box[2]
        xmax = go_box[3]
        x_center = ((xmax - xmin)/2)+xmin
        y_center = ((ymax - ymin)/2)+ymin
        print(y_center, x_center)
        turn_dir = 10
        drive_dir = 10
        if x_center > 0.5: # TURN right --> (+)
            turn_dir*=-1
        if y_center > 0.5: # MOVE backwards
            drive_dir*=-1
        robot.turn_in_place(degrees(turn_dir)).wait_for_completed()
        robot.drive_straight(distance_mm(drive_dir), speed_mmps(20)).wait_for_completed()
        robot.wait_for_all_actions_completed()
        
    print("Oriented to object!")
    robot.wait_for_all_actions_completed()

In [ ]:
import threading
from classes.vision import Vision

second_look = Vision(cozmo.robot.Robot)

threading.Thread(target=second_look.detectImages).start()

cozmo.run_program(cozmo_program, use_viewer=False)




Detect Images started


2019-10-21 11:20:59,593 cozmo.general INFO     App connection established. sdk_version=1.4.11.dev0 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2019-10-21 11:20:59,656 cozmo.general INFO     Found robot id=1
2019-10-21 11:20:59,769 cozmo.general INFO     Connected to Android device serial=03160355293b2004
2019-10-21 11:20:59,875 cozmo.general INFO     Robot id=1 serial=024086a7 initialized OK


_____

In [ ]:

# l_d = xmin
# r_d = 1.0 - xmax 
# t_d = ymin
# b_d = 1.0 - ymax
# print('current head angle?', robot._head_angle)
# if r_d > l_d:
#     # MOVE HEAD LEFT
#     # move 1/2 * r_d

# else:
#     # MOVE HEAD RIGHT
#     # move head 1/2 * l_d

# if b_d > t_d:
#     # MOVE HEAD UP
#     # move head 1/2 * b_d

# else:
#     # MOVE HEAD DOWN
#     # move head 1/2 * t_d   

In [ ]:
def test_box(box):
    vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            box['detection_boxes'],
            box['detection_classes'],
            box['detection_scores'],
            category_index,
#           instance_masks=output_dict.get('detection_masks'),
            use_normalized_coordinates=True,
            min_score_thresh=.10,
            line_thickness=8)

________________________

In [ ]:
import sys
sys.path.append('/home/david/GIT/language-acquisition')
from language_acquisition import cozmo_program
# look = Sight(robot)

In [ ]:
def turn_towards_face(self, face, in_parallel=False, num_retries=0):
    '''Tells Cozmo to turn towards this face.

    Args:
        face: (:class:`cozmo.faces.Face`): The face Cozmo will turn towards.
        in_parallel (bool): True to run this action in parallel with
            previous actions, False to require that all previous actions
            be already complete.
        num_retries (int): Number of times to retry the action if the
            previous attempt(s) failed.
    Returns:
        A :class:`cozmo.robot.TurnTowardsFace` action object which can be
            queried to see when it is complete
    '''
    action = self.turn_towards_face_factory(face=face,
            conn=self.conn, robot=self, dispatch_parent=self)
    self._action_dispatcher._send_single_action(action,
                                                in_parallel=in_parallel,
                                                num_retries=num_retries)
    return action

In [ ]:
def turn_towards_object(self, obj, in_parallel=False, num_retries=0):
     '''Tells Cozmo to turn towards that object.

    Args:
        obj: (...`): The object Cozmo will turn towards.
        in_parallel (bool): True to run this action in parallel with
            previous actions, False to require that all previous actions
            be already complete.
        num_retries (int): Number of times to retry the action if the
            previous attempt(s) failed.
    Returns:
        A :class:`cozmo.robot.TurnTowardsFace` action object which can be
            queried to see when it is complete
    '''
    action = self.turn_towards_face_factory(face=face,
            conn=self.conn, robot=self, dispatch_parent=self)
    self._action_dispatcher._send_single_action(action,
                                                in_parallel=in_parallel,
                                                num_retries=num_retries)
    return action

In [ ]:
import asyncio
import time

In [ ]:
# find that code that commanded Cozmo to "trundle" in search of bounding box

obj = None

while not obj:
    # roll wheels; one more than the other. 
    robot.
    
    try:
        # sweep head medium to down; 
        
        # drive in circles;
        
        # avoid edges
        
    # move head DOWN and lift UP
        if bounding_box:
            print("Found object!")
            obj = bounding_box
            robot.wait_for_all_actions_completed()
    except asyncio.TimeoutError:
        print("Didn't find an object - exiting!")
        return
    
    time.sleep(.1)
    

### select biggest bounding box

### re-orient

target_object (:class:`cozmo.objects.LightCube`): The cube to roll.

target_object (:class:`cozmo.objects.ObservableObject`): The destination object. CustomObject instances are not supported.   

*What is "**kw"?*

In [ ]:
class FixedCustomObject():
    '''A fixed object defined by the SDK. It is given a pose and x,y,z sizes.

    This object cannot be observed by the robot so its pose never changes.
    The position is static in Cozmo's world view; once instantiated, these
    objects never move. This could be used to make Cozmo aware of objects and
    know to plot a path around them even when they don't have any markers.
    
    To create these use :meth:`~cozmo.world.World.create_custom_fixed_object`
    '''

    is_visible = False

    def __init__(self, pose, x_size_mm, y_size_mm, z_size_mm, object_id, *a, **kw):
        super().__init__(*a, **kw)
        self._pose = pose
        self._object_id = object_id
        self._x_size_mm = x_size_mm
        self._y_size_mm = y_size_mm
        self._z_size_mm = z_size_mm

    def __repr__(self):
        return ('<%s pose=%s object_id=%d x_size_mm=%.1f y_size_mm=%.1f z_size_mm=%.1f=>' %
                                        (self.__class__.__name__, self.pose, self.object_id,
                                         self.x_size_mm, self.y_size_mm, self.z_size_mm))

    #### Private Methods ####
    #### Event Wrappers ####
    #### Properties ####
    @property
    def object_id(self):
        '''int: The internal ID assigned to the object.

        This value can only be assigned once as it is static in the engine.
        '''
        return self._object_id

    @object_id.setter
    def object_id(self, value):
        if self._object_id is not None:
            raise ValueError("Cannot change object ID once set (from %s to %s)" % (self._object_id, value))
        logger.debug("Updated object_id for %s from %s to %s", self.__class__, self._object_id, value)
        self._object_id = value

    @property
    def pose(self):
        ''':class:`cozmo.util.Pose`: The pose of the object in the world.'''
        return self._pose

    @property
    def x_size_mm(self):
        '''float: The length of the object in its X axis, in millimeters.'''
        return self._x_size_mm

    @property
    def y_size_mm(self):
        '''float: The length of the object in its Y axis, in millimeters.'''
        return self._y_size_mm

    @property
    def z_size_mm(self):
        '''float: The length of the object in its Z axis, in millimeters.'''
        return self._z_size_mm